In [1]:
from pyspark.sql import SparkSession
import pandas as pd
import os
import pyspark.sql.functions as sf      # sf = spark functions
import pyspark.sql.types as st          # st = spark types

#create Spark Session
spark = SparkSession.builder.getOrCreate()

your 131072x1 screen size is bogus. expect trouble


23/02/10 09:38:30 WARN Utils: Your hostname, DESKTOP-SU1U0NI resolves to a loopback address: 127.0.1.1; using 172.21.251.109 instead (on interface eth0)
23/02/10 09:38:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/10 09:38:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [25]:
# file path
data_dir = "./data"
file_name = 'coffee.csv'

# define schema and read file into DataFrame
schema = "Date date, Open float, High float, Low float, Close float, Volume float, Currency string"
df = spark.read.csv(os.path.join(data_dir,file_name), schema=schema, header=True, enforceSchema=True)

# rename all columns to lower case
for col in df.columns:
  df = df.withColumnRenamed(col, col.lower())

# calculate (open - close), (high-low), volume
check_daily_volume = sf.udf(lambda x: True if x > 100 else False)

df = df.withColumn('open_close', (sf.col('open') - sf.col('close')))
df = df.withColumn('high_low', (sf.col('high') - sf.col('low')).cast('float'))
df = df.withColumn('check_volume', check_daily_volume(sf.col('volume')))

df.show(10)

+----------+------+------+------+------+-------+--------+----------+---------+------------+
|      date|  open|  high|   low| close| volume|currency|open_close| high_low|check_volume|
+----------+------+------+------+------+-------+--------+----------+---------+------------+
|2000-01-03|122.25| 124.0| 116.1| 116.5| 6640.0|     USD|      5.75|7.9000015|        true|
|2000-01-04|116.25| 120.5|115.75|116.25| 5492.0|     USD|       0.0|     4.75|        true|
|2000-01-05| 115.0| 121.0| 115.0| 118.6| 6165.0|     USD|-3.5999985|      6.0|        true|
|2000-01-06| 119.0| 121.4| 116.5|116.85| 5094.0|     USD| 2.1500015|4.9000015|        true|
|2000-01-07|117.25|117.75| 113.8|114.15| 6855.0|     USD| 3.0999985| 3.949997|        true|
|2000-01-10| 123.5| 126.0| 116.7|117.55| 7499.0|     USD|  5.949997| 9.300003|        true|
|2000-01-11| 115.5|118.25| 115.5| 117.8| 3976.0|     USD| -2.300003|     2.75|        true|
|2000-01-12| 117.8| 120.5| 116.9|118.95| 5184.0|     USD|-1.1499939|3.5999985|  